In [13]:
import numpy as np
import cv2
import sys
import os
import dlib
subjects = ["", "Nicke"]
from imutils import face_utils
from scipy.spatial import distance as dist
import imutils
import winsound

In [14]:
def detect_face(img):
#convert the test image to gray scale as opencv face detector expects gray images
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    

#there is also a more accurate but slow: Haar classifier
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
 
#let's detect multiscale images(some images may be closer to camera than others)
#result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
#if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
 
#under the assumption that there will be only one face,
#extract the face area
    (x, y, w, h) = faces[0]
 
#return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

In [15]:
#and will return two lists of exactly same size, one list 
#of faces and another list of labels for each face
def prepare_training_data(data_folder_path):
 
#------STEP-1--------
#get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
 
#list to hold all subject faces
    faces = []
#list to hold labels for all subjects
    labels = []
 
#let's go through each directory and read images within it
    for dir_name in dirs:
 
#our subject directories start with letter 's' so
#ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;
 
#------STEP-2--------
#extract label number of subject from dir_name
#format of dir name = slabel
#, so removing letter 's' from dir_name will give us label
    label = int(dir_name.replace("s", ""))
 
#build path of directory containing images for current subject subject
#sample subject_dir_path = "training-data/s1"
    subject_dir_path = data_folder_path + "/" + dir_name
 
#get the images names that are inside the given subject directory
    subject_images_names = os.listdir(subject_dir_path)
 
#------STEP-3--------
#go through each image name, read image, 
#detect face and add face to list of faces
    for image_name in subject_images_names:
 
#ignore system files like .DS_Store
        #if image_name.startswith("."):
        #continue;
 
#build image path
#sample image path = training-data/s1/1.pgm
        image_path = subject_dir_path + "/" + image_name
 
#read image
        image = cv2.imread(image_path)
 
#display an image window to show the image 
        cv2.imshow("Training on image...", image)
        cv2.waitKey(100)
 
#detect face
        face, rect = detect_face(image)
 
#------STEP-4--------
#for the purpose of this tutorial
#we will ignore faces that are not detected
        if face is not None:
#add face to list of faces
            faces.append(face)
#add label for this face
        labels.append(label)
 
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
 
    return faces, labels

In [16]:
#let's first prepare our training data
#data will be in two lists of same size
#one list will contain all the faces
#and the other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")
 
#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))


Preparing data...
Data prepared
Total faces:  10
Total labels:  10


In [17]:
#create our LBPH face recognizer 
face_recognizer =cv2.face.LBPHFaceRecognizer_create()

In [18]:
face_recognizer.train(faces, np.array(labels))

In [19]:
#function to draw rectangle on image 
#according to given (x, y) coordinates and 
#given width and heigh
def draw_rectangle(img, rect):
 (x, y, w, h) = rect
 cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
 
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
 cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [20]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
 
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
 
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
 
    # return the eye aspect ratio
    return ear

In [21]:
#this function recognizes the person in image passed
#and draws the name of the subject 
#allso draws 68 facial landmarks and uses the eye landmarks to determine if eyes
#open or shut.
def predict(test_img):
    global framecounter
    global eyesOpen
    global eyesClosed
#make a copy of the image as we don't want to change original image
    img = test_img.copy()
#detect face from the image
    face, rect = detect_face(img)
    
    rects = detector(img,0)
    for (i, rect) in enumerate(rects):
        shape = predictor(img, rect)
        shape = face_utils.shape_to_np(shape)
        for (x, y) in shape:
            cv2.circle(img, (x, y), 2, (0,255,0), -1)
            
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    #ear = a value for the "open-ness of the eyes"
    ear = (leftEAR + rightEAR) / 2.0
    #found that an ear value belov 0.3 = closed eyes
    if ear < 0.3:
        #add 2 counters (one for alert and one for PERCLOS) if eyes are
        #considered closed to count the number 
        #of frames (ear value less then 0.3)
        framecounter +=1
        eyesClosed +=1
        draw_text(img, "eyes closed", x-200, y-250)
        
    elif ear >0.3:
        eyesOpen +=1
        framecounter =0
        draw_text(img, "eyes open", x-200, y-250)
    print (ear)
    print (framecounter)
    #a blinck checker (a blink is less than 7 frames long.)
    #if not a blink, plays a "wakeup sound"
    if framecounter >=7:
        winsound.PlaySound('airHorn.wav', winsound.SND_FILENAME)
        framecounter = 0
    #calculates the % of time that eyes are closed, can be
    #an indicator of tiredness.
    PERCLOS = (eyesClosed/(eyesOpen+eyesClosed)) * 100
    #because PERCLOSE is't a value untill the eyes has been closed atleast
    #once, this if statement is needed.
    if PERCLOS >0:
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, "PERCLOS "+str(PERCLOS) ,(500,700), font, 1,(0,0,255),2,cv2.LINE_AA)
    
            
#predict the image using our face recognizer 
    label = face_recognizer.predict(face)
#get name of respective label returned by face recognizer
    label_text = subjects[label[0]]

    
#draw name of predicted person
    
    draw_text(img, label_text, x-200, y-200)
 
    return img

In [22]:
print("runing...")

global framecounter
framecounter = 0
eyesOpen =0
eyesClosed= 0
p = "shape_predictor_68_face_landmarks.dat"
#start dlib's face detector and create the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)
#read the video file
video_capture = cv2.VideoCapture('test.mp4')
 
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    #run the frame through the prediction function
    predicted_img = predict(frame)
    #display the name of the detected person
    cv2.imshow(subjects[0], predicted_img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video_capture.release()
cv2.destroyAllWindows()

runing...
0.3503412052586733
0
0.361321330276142
0
0.35506518622443906
0
0.33232336300160803
0
0.35738620125994564
0
0.34684225903851795
0
0.35973705595437927
0
0.3591658056949335
0
0.36240853662917466
0
0.3454064289613801
0
0.34361151675596624
0
0.344540029027672
0
0.3449235511502659
0
0.3533264214621678
0
0.3545271653667467
0
0.3467669729066155
0
0.33896480881993396
0
0.3421446767802615
0
0.34233314456372255
0
0.35144343400371714
0
0.33907158070974475
0
0.33060113678438896
0
0.3337239955278546
0
0.33052048920781313
0
0.3413383401138585
0
0.3379335987223604
0
0.3331924014828458
0
0.33386557787498095
0
0.35935984648520547
0
0.3565262039756687
0
0.36160143137886325
0
0.373530366362295
0
0.35360757421144906
0
0.36381575438062097
0
0.3650287263079006
0
0.3725092655682406
0
0.36022999713682524
0
0.36323090527520574
0
0.33628622485974763
0
0.16946630475287328
1
0.15905473891863606
2
0.2184719730813835
3
0.28619420478156454
4
0.29774064458525273
5
0.30016909084421184
0
0.30563929667213285
0
